# Maps for announcement blog post

Visualization 2: The carbon intensity of consumed electricity differs from generated electricity
* Show a static carbon flow map focused on a single BA plus all directly-interconnected BAs
* Pick an hour when there is some particularly dirty electricity getting imported
* Each BA would be represented by a bubble, where the color changes based on carbon intensity, and carbon flows would be represented by colored arrows between the bubbles. 
* To illustrate the difference between produced and consumed, we might want to have a pair of bubbles for each BA - one that shows the produced CI and one that shows the consumed CI. If we do this, we probably don’t want to vary the size of each bubble based on total generation. Or maybe we could do a split bubble - the top half shows produced CI and the bottom shows consumed CI.


Visualization 3: Animating hourly and consumed emissions for the whole county
* This animation should put the previous two concepts together and show how carbon flows and how CI changes across the entire country for a single day (or a week?)
* We could also potentially have two maps side by side: one that shows annual averages in bubbles (with no carbon flow), and one that shows the animated hourly flow (to really draw the distinction between annual and hourly datasets)


### Ref for making gif: 
`https://stackoverflow.com/questions/753190/programmatically-generate-video-or-animated-gif-in-python`

In [2]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import * 
import plotly.io as pio
import os
import pandas as pd
import numpy as np


In [3]:
%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys

sys.path.append("../../src/")

import output_data

In [4]:
ba_coords = pd.read_csv("resources/ba_coords.csv", index_col=0, dtype={"cx":np.float64, "cy":np.float64})
ba_meta = pd.read_csv("../../data/manual/ba_reference.csv", index_col=0)

In [5]:
# Note: 150+ BAs are not in ba_coords
ba_list = ba_meta[(ba_meta.ba_category != "misellaneous") & (ba_meta.us_ba) & (ba_meta.index.isin(ba_coords.index))].index
ba_list = [ba for ba in ba_list if (f"{ba}.csv" in os.listdir("../../data/results/2020/power_sector_data/hourly/us_units/"))]

In [6]:
hour = "2020-08-01T00:00+00"
range_start = "2020-08-01T00:00+00"
range_end = "2020-08-7T23:00+00"

In [7]:
cleaned_io = pd.read_csv("../../data/outputs/2020/eia930/eia930_elec.csv", index_col=0, parse_dates=True)
cleaned_io = cleaned_io[[c for c in cleaned_io.columns if ".ID." in c]]

In [8]:
ID_COL = "EBA.%s-%s.ID.H"

In [9]:
all = []
for ba in ba_list:
    produced = pd.read_csv(f"../../data/results/2020/power_sector_data/hourly/us_units/{ba}.csv", index_col="datetime_utc", parse_dates=True, usecols=["datetime_utc","fuel_category", "net_generation_mwh", "generated_co2_rate_lb_per_mwh_for_electricity"])
    produced = produced[produced.fuel_category == "total"]
    produced = produced.drop(columns=["fuel_category"])
    
    if ba_meta.loc[ba,"ba_category"] == "generation_only":
        consumed = pd.DataFrame(index=produced.index, columns=[["consumed_co2_rate_lb_per_mwh_for_electricity"]], dtype=np.float64)
    else:
        consumed = pd.read_csv(f"../../data/results/2020/carbon_accounting/hourly/us_units/{ba}.csv", index_col="datetime_utc", parse_dates=True, usecols=["datetime_utc", "consumed_co2_rate_lb_per_mwh_for_electricity"])
    consumed.columns = consumed.columns.get_level_values(0)

    both = pd.concat([produced, consumed], axis='columns')
    #both = both.loc[range_start:range_end]
    both = both.reset_index()
    both["BA"] = ba
    all.append(both)

all = pd.concat(all)

In [10]:
# Add coordinates
all = all.merge(ba_coords, how='left', validate='many_to_one', left_on="BA", right_index=True)

In [11]:
tester = (all[all.BA=="CISO"]).copy()
tester["difference"] = tester.consumed_co2_rate_lb_per_mwh_for_electricity - tester.generated_co2_rate_lb_per_mwh_for_electricity
tester.difference.abs().max()
tester[tester.difference == tester.difference.abs().max()]

,datetime_utc,net_generation_mwh,generated_co2_rate_lb_per_mwh_for_electricity,consumed_co2_rate_lb_per_mwh_for_electricity,BA,cx,cy,difference
195,2020-01-09 11:00:00+00:00,13151.14,485.225459,628.245034,CISO,60.0,270.0,143.019575


In [12]:
# Hour with max CISO difference 
hour = '2020-09-25 12:00:00+00:00'

In [13]:
hours = all.datetime_utc[(all.datetime_utc < pd.to_datetime(range_end)) & (all.datetime_utc > pd.to_datetime(range_start)) & (all.BA == "CISO")]
for hour in hours:
    print(hour, end="...")
    io_toplot = cleaned_io.loc[hour]
    toplot = all[all.datetime_utc == hour]
    fig = go.Figure()

    toplot.loc[toplot.net_generation_mwh < 1, "net_generation_mwh"] = 1
    sizes = np.log(toplot.net_generation_mwh)/np.log(1.5)
    offset = sizes/2.5
    fig.add_trace(
        go.Scatter(x=toplot.cx, y=toplot.cy, mode="markers", hoverinfo="text", text=toplot.BA, 
            marker_symbol="triangle-ne",
            marker=dict(color=toplot.consumed_co2_rate_lb_per_mwh_for_electricity, size=sizes,
                sizemode='diameter', cmin=0, cmax=1800, colorbar=dict(
                title="Emission rate (lbs/MWh)", orientation='h'
            ),
            colorscale="YlOrRd"),
            name="Consumed",
            
        )
        
    )
    fig.add_trace(
        go.Scatter(x=toplot.cx-offset, y=toplot.cy+offset, mode="markers", hoverinfo="text", text=toplot.BA, 
            marker_symbol="triangle-sw", 
            marker=dict(color=toplot.generated_co2_rate_lb_per_mwh_for_electricity, size=sizes,
                sizemode='diameter', cmin=0, cmax=1800,
            colorscale="YlOrRd"),
            name="Generated"
            
        )
        
    )
    fig.update_yaxes(range=(550,0)) # autorange="reversed")
    fig.update_xaxes(range=(0,800))

    # max_width = io_toplot.max()
    # width_factor = 8/max_width
    width_factor = 1/200
    for name, val in io_toplot.iteritems():
        if val <= 0: 
            continue 
        bas = name.split(".")[1].split("-")
        (ba1, ba2) = bas

        next = False
        for ba in bas: 
            if ba not in ba_coords.index:
                next=True
        if next:
            continue
        
        # Arrows have to be added separately
        line_size = val*width_factor
        fig.add_annotation(
            x=ba_coords.loc[ba2,"cx"],  # arrows' head
            y=ba_coords.loc[ba2,"cy"],  # arrows' head
            ax=ba_coords.loc[ba1,"cx"],  # arrows' tail
            ay=ba_coords.loc[ba1,"cy"],  # arrows' tail
            xref='x',
            yref='y',
            axref='x',
            ayref='y',
            text='',  # if you want only the arrow
            showarrow=True,
            arrowhead=1,
            arrowsize=1, #max(.3, line_size),
            arrowwidth=1,
            arrowcolor='royalblue'
        )

        # Lines on top of arrows 
        fig.add_trace(
            go.Scatter(x = ba_coords.loc[bas,"cx"], y = ba_coords.loc[bas,"cy"], 
                mode="lines", line = dict(color='royalblue', width=1), showlegend=False
            )
        )

    # Add images
    fig.add_layout_image(
            dict(
                source="resources/usa.png",
                xref="x",
                yref="y",
                x=10,
                y=0,
                sizex=790,
                sizey=550,
                sizing="stretch",
                opacity=0.5,
                layer="below")
    )

    # Set templates
    fig.update_layout(template="plotly_white", width=800, height=600)
    #fig.show()
    pio.write_image(fig, f"outputs/maps/{hour}.png")


2020-08-01 01:00:00+00:00...2020-08-01 02:00:00+00:00...2020-08-01 03:00:00+00:00...2020-08-01 04:00:00+00:00...2020-08-01 05:00:00+00:00...2020-08-01 06:00:00+00:00...2020-08-01 07:00:00+00:00...2020-08-01 08:00:00+00:00...2020-08-01 09:00:00+00:00...2020-08-01 10:00:00+00:00...2020-08-01 11:00:00+00:00...2020-08-01 12:00:00+00:00...2020-08-01 13:00:00+00:00...2020-08-01 14:00:00+00:00...2020-08-01 15:00:00+00:00...2020-08-01 16:00:00+00:00...2020-08-01 17:00:00+00:00...2020-08-01 18:00:00+00:00...2020-08-01 19:00:00+00:00...2020-08-01 20:00:00+00:00...2020-08-01 21:00:00+00:00...2020-08-01 22:00:00+00:00...2020-08-01 23:00:00+00:00...2020-08-02 00:00:00+00:00...2020-08-02 01:00:00+00:00...2020-08-02 02:00:00+00:00...2020-08-02 03:00:00+00:00...2020-08-02 04:00:00+00:00...2020-08-02 05:00:00+00:00...2020-08-02 06:00:00+00:00...2020-08-02 07:00:00+00:00...2020-08-02 08:00:00+00:00...2020-08-02 09:00:00+00:00...2020-08-02 10:00:00+00:00...2020-08-02 11:00:00+00:00...2020-08-02 12:00:00+

In [14]:
# Now just CISO and neighbors. 

# Get list of CISO neighbors 
ciso_interchanges = [c for c in cleaned_io.columns if "CISO" in c]
ciso_bas = []
for ci in ciso_interchanges: 
    ba1, ba2 = ci.split(".")[1].split("-")
    if ba1 not in ciso_bas: 
        ciso_bas.append(ba1)
    if ba2 not in ciso_bas:
        ciso_bas.append(ba2)



In [15]:
print(ciso_interchanges)
print(ciso_bas)

['EBA.AZPS-CISO.ID.H', 'EBA.BANC-CISO.ID.H', 'EBA.BPAT-CISO.ID.H', 'EBA.CEN-CISO.ID.H', 'EBA.CISO-AZPS.ID.H', 'EBA.CISO-BANC.ID.H', 'EBA.CISO-BPAT.ID.H', 'EBA.CISO-CEN.ID.H', 'EBA.CISO-IID.ID.H', 'EBA.CISO-LDWP.ID.H', 'EBA.CISO-NEVP.ID.H', 'EBA.CISO-PACW.ID.H', 'EBA.CISO-SRP.ID.H', 'EBA.CISO-TIDC.ID.H', 'EBA.CISO-WALC.ID.H', 'EBA.IID-CISO.ID.H', 'EBA.LDWP-CISO.ID.H', 'EBA.NEVP-CISO.ID.H', 'EBA.PACW-CISO.ID.H', 'EBA.SRP-CISO.ID.H', 'EBA.TIDC-CISO.ID.H', 'EBA.WALC-CISO.ID.H']
['AZPS', 'CISO', 'BANC', 'BPAT', 'CEN', 'IID', 'LDWP', 'NEVP', 'PACW', 'SRP', 'TIDC', 'WALC']


In [16]:
# Hour with max CISO difference generated / consumed 
hour = '2020-09-25 12:00:00+00:00'

In [31]:
# src: https://community.plotly.com/t/how-to-include-a-colorscale-for-color-of-line-graphs/38002/3 
from ast import literal_eval
def get_color_for_val(val, vmin, vmax, pl_colors):
    if pl_colors[0][:3] != 'rgb':
        raise ValueError('This function works only with Plotly  rgb-colorscales')
    if vmin >= vmax:
        raise ValueError('vmin should be < vmax')

    scale = [round(k / (len(pl_colors)), 3) for k in range(len(pl_colors) + 1)]

    colors_01 = np.array([literal_eval(color[3:]) for color in pl_colors]) / 255  # color codes in [0,1]

    v = (val - vmin) / (vmax - vmin)  # val is mapped to v in [0,1]
    # find two consecutive values in plotly_scale such that   v is in  the corresponding interval
    idx = 0

    while (v > scale[idx + 1]):
        idx += 1

    vv = (v - scale[idx]) / (scale[idx + 1] - scale[idx])

    # get   [0,1]-valued color code representing the rgb color corresponding to val
    if idx == len(pl_colors)-1: # Make this work when some values exceed range
        val_color01 = colors_01[idx] # color by last color 
    else: 
        val_color01 = colors_01[idx] + vv * (colors_01[idx + 1] - colors_01[idx])

    val_color_0255 = (255 * val_color01 + 0.5).astype(int)
    return f'rgb{str(tuple(val_color_0255))}'

In [62]:
named_colorscales()

['aggrnyl',
 'agsunset',
 'blackbody',
 'bluered',
 'blues',
 'blugrn',
 'bluyl',
 'brwnyl',
 'bugn',
 'bupu',
 'burg',
 'burgyl',
 'cividis',
 'darkmint',
 'electric',
 'emrld',
 'gnbu',
 'greens',
 'greys',
 'hot',
 'inferno',
 'jet',
 'magenta',
 'magma',
 'mint',
 'orrd',
 'oranges',
 'oryel',
 'peach',
 'pinkyl',
 'plasma',
 'plotly3',
 'pubu',
 'pubugn',
 'purd',
 'purp',
 'purples',
 'purpor',
 'rainbow',
 'rdbu',
 'rdpu',
 'redor',
 'reds',
 'sunset',
 'sunsetdark',
 'teal',
 'tealgrn',
 'turbo',
 'viridis',
 'ylgn',
 'ylgnbu',
 'ylorbr',
 'ylorrd',
 'algae',
 'amp',
 'deep',
 'dense',
 'gray',
 'haline',
 'ice',
 'matter',
 'solar',
 'speed',
 'tempo',
 'thermal',
 'turbid',
 'armyrose',
 'brbg',
 'earth',
 'fall',
 'geyser',
 'prgn',
 'piyg',
 'picnic',
 'portland',
 'puor',
 'rdgy',
 'rdylbu',
 'rdylgn',
 'spectral',
 'tealrose',
 'temps',
 'tropic',
 'balance',
 'curl',
 'delta',
 'oxy',
 'edge',
 'hsv',
 'icefire',
 'phase',
 'twilight',
 'mrybm',
 'mygbm']

In [86]:
io_toplot = cleaned_io.loc[hour, ciso_interchanges]
toplot = all[all.datetime_utc == hour]
toplot = toplot[toplot.BA.isin(ciso_bas)]
fig = go.Figure()

colorscale = diverging.RdYlGn_r
#colorscale = cmocean.solar_r

c_max = np.floor(toplot.generated_co2_rate_lb_per_mwh_for_electricity.max() + 100)

### From when 
# max_width = io_toplot.max()
# width_factor = 8/max_width
#width_factor = 1/200
for name, val in io_toplot.iteritems():
    if val <= 0: 
        continue 
    bas = name.split(".")[1].split("-")
    (ba1, ba2) = bas

    next = False
    for ba in bas: 
        if ba not in ba_coords.index:
            next=True
        if ba not in toplot.BA.unique():
            next=True
    if next:
        continue

    color = toplot.loc[toplot.BA == ba1, "generated_co2_rate_lb_per_mwh_for_electricity"].to_numpy()[0]
    print(color)

    fig.add_trace(
        go.Scatter(x = ba_coords.loc[bas,"cx"], y = ba_coords.loc[bas,"cy"], opacity=1.0,
            mode="lines", line = dict(color=get_color_for_val(color, 0, c_max, colorscale), width=2), showlegend=False
        )
    )

################################# Plot BAs 
toplot.loc[toplot.net_generation_mwh < 1, "net_generation_mwh"] = 1
sizes = np.log(toplot.net_generation_mwh)/np.log(1.5)
offset = sizes/1.6
fig.add_trace(
    go.Scatter(x=toplot.cx, y=toplot.cy-offset, mode="markers", hoverinfo="text", text=toplot.BA, 
        marker_symbol="triangle-up", 
        opacity=1.0,
        marker=dict(color=toplot.generated_co2_rate_lb_per_mwh_for_electricity, size=sizes,
            sizemode='diameter', cmin=0, cmax=c_max, opacity=1.0,
        colorscale="rdylgn_r"),
        name="Generated",   
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(x=toplot.cx, y=toplot.cy, mode="markers", hoverinfo="text", text=toplot.BA, 
        marker_symbol="triangle-down",
        marker=dict(color=toplot.consumed_co2_rate_lb_per_mwh_for_electricity, size=sizes, opacity=1.0,
            sizemode='diameter', cmin=0, cmax=c_max, colorbar=dict(
            title="Emission rate (lbs/MWh)", orientation='v', len=.8, thickness=20, yanchor='bottom', y=0, xpad=20
        ),
        colorscale="rdylgn_r"),
        name="Consumed", 
        showlegend=False  
    )
)


# Legends: don't want colored markers
# Legends: don't want colored markers
fig.add_trace(
    go.Scatter(x=[-10], y=[-10], mode="markers", 
        marker_symbol="triangle-up",
        marker=dict(color='white', line=dict(width=2, color='DarkSlateGrey'), size=10),
        name="Generated",   
    )
)
fig.add_trace(
    go.Scatter(x=[-10], y=[-10], mode="markers", 
        marker_symbol="triangle-down",
        marker=dict(color='white', line=dict(width=2, color='DarkSlateGrey'), size=10),
        name="Consumed",   
    )
)
fig.update_yaxes(range=(420,0)) # autorange="reversed")
fig.update_xaxes(range=(0,200))

## loop through the labels and add them as annotations
for x in zip(toplot.BA, toplot.cx, toplot.cy):
    left_bas = ["BANC","TIDC","CISO","LDWP","IID"]
    delta = (-12 if x[0] in left_bas else 12)
    fig.add_annotation(
        x=x[1] + delta,
        y=x[2],
        text=x[0],
        showarrow=False,
        xanchor=('right' if x[0] in left_bas else 'left')
    )

# Add images
fig.add_layout_image(
        dict(
            source="resources/usa.png",
            xref="x",
            yref="y",
            x=10,
            y=0,
            sizex=790,
            sizey=550,
            sizing="stretch",
            opacity=1.0,
            layer="below")
)

# Set templates
fig.update_layout(template="plotly_white", width=450, height=500,
    yaxis_visible=False, xaxis_visible=False
)
fig.show()
fig.write_image(f"outputs/viz2_label_{hour}.png") 

2051.357942055572
791.7235362136025
494.285163780544
486.6971936773157
468.0531456179542
1171.858214304538
873.4998685340672
725.9808186972756
640.9137194410688
742.8031879262338


In [76]:
diverging.RdYlGn_r

['rgb(0,104,55)',
 'rgb(26,152,80)',
 'rgb(102,189,99)',
 'rgb(166,217,106)',
 'rgb(217,239,139)',
 'rgb(255,255,191)',
 'rgb(254,224,139)',
 'rgb(253,174,97)',
 'rgb(244,109,67)',
 'rgb(215,48,39)',
 'rgb(165,0,38)']

In [87]:
import kaleido

In [ ]:
### Old code for arrows
    # # Arrows have to be added separately
    # line_size = val*width_factor
    # fig.add_annotation(
    #     x=ba_coords.loc[ba2,"cx"],  # arrows' head
    #     y=ba_coords.loc[ba2,"cy"],  # arrows' head
    #     ax=ba_coords.loc[ba1,"cx"],  # arrows' tail
    #     ay=ba_coords.loc[ba1,"cy"],  # arrows' tail
    #     xref='x',
    #     yref='y',
    #     axref='x',
    #     ayref='y',
    #     text='',  # if you want only the arrow
    #     showarrow=True,
    #     arrowhead=1,
    #     arrowsize=1, #max(.3, line_size),
    #     arrowwidth=1,
    #     arrowcolor='royalblue'
    # )

In [32]:
import imageio
images = []
for f in os.listdir("../visualization_output/maps/"):
    if ".png" not in f:
        continue
    images.append(imageio.imread("../visualization_output/maps/"+f))
imageio.mimsave("../visualization_output/movie.gif", images)

/var/folders/mr/3dcnnc3153j6s05bmdwr0r0h0000gq/T/ipykernel_4357/4049740564.py:6: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.

